# Extracting Data from AWS and removing duplicates.

Bringing data of AWS and onto local machine, removing any duplicate entries along the way.

In [2]:
# need dat pymongo
!pip install pymongo

    100% |████████████████████████████████| 378kB 1.5MB/s eta 0:00:01


In [1]:
# make sure the figgin MongoDB Container is on 
import pandas as pd

# Actual IP and port changed
from pymongo import MongoClient
client = MongoClient('250.250.250.250',20000)

In [2]:
import json

In [3]:
# get the collection
Beers = client['Beers']

In [7]:
# this is collection i need to query. 
Beers['Beer_final']

Collection(Database(MongoClient(host=['54.187.99.153:27016'], document_class=dict, tz_aware=False, connect=True), 'Beers'), 'Beer_final')

In [8]:
beer = Beers['Beer_final'].find_one({})
beer_name, brewery_name = beer['Beer_Name'], beer['Brewery_Name']
print(beer_name, brewery_name)

Indian Brown Ale Dogfish Head Craft Brewery


### First things first: Remove duplicate entries.
- Figure out how to query mongodb
- find a way to set some of the extracted information.


In [32]:
# all beer names that I had in the DB.
Beer_names = [post['Beer_Name'] for post in Beers['Beer_final'].find({})]

In [55]:
Beer_combos = [(post['Beer_Name'],post['Brewery_Name']) for post in Beers['Beer_final'].find({})]

In [40]:
# how many actual beers there are to compare. 
len(set(Beer_names))

9311

In [56]:
# how many beer/Brewery Combos.
len(set(Beer_combos))

9490

In [38]:
# get a beer name
Beer_names[1]

'Palo Santo Marron'

In [39]:
# see if this shit works.
Beers['Beer_final'].find_one({'Beer_Name':'Palo Santo Marron'})

{'1': '. Roastiness also there and well constrained. Six months later, room temperature, more chocolate, thick and woody.\xa0',
 '10': 'Both my girlfriend, and a good friend of mine, sent that article about DFH from the New Yorker to me. Of course I had already read it via BeerAdvocate, but appreciated the gestures nonetheless. So when I saw this on the shelf in downtown Vancouver last month, I procured a bottle for each of them (to share with me, of course). The first one vanished none too slowly, so I had better write this one down...\n\nThis beer pours a deep, dark black, with reddish tints, and a single finger of tan head, that disappears readily. The smell is prunes, dates, and an intense wood essence, that I admit I cannot identify (beyond the description on the label). The taste is floral hops, heavy black fruit, caramel, vanilla, and an acrid woodiness, again hard to describe. The carbonation is quite low, and the finish carries on the heavy woody sweetness. The body is very th

In [67]:
# for loop, goes through all unique names
for beerio in Beer_combos[1]:
    CB = Beers['Beer_final'].find_one({'Beer_Name':beerio})
    #dict(CB)
    print(dict(CB))
    #Beers['Beers_Unique'].insert_one(CB)

{'_id': ObjectId('5a629bad79e18b002c0bb9aa'), '1': '. Roastiness also there and well constrained. Six months later, room temperature, more chocolate, thick and woody.\xa0', '2': 'A: two finger milk chocolate head on a jet black body which poured out like oil.\n\nS: vanilla, caramel, toffee, butter, earthy hops and wood all prevail with a boozy undertone.\n\nT: I was expecting the worst but this one was as boozy as I was expecting from the 12% ABV. Starts with vanilla and caramel moving to wood then a chocolate and roasted malt finish with dryness provided by mildly bitter coffee. \n\nM: thick and oily with moderate carbonation.\n\nOverall: the booze are well hidden. This one is actually well balanced. I think the wood cleans this one up and smooths it up.\xa0', '3': "Pours an opaque super dark brown (almost black really) with a foamy dark khaki head that settles to an oily film on top of the beer. Small streaks of lace form around the glass on the drink down. Smell is of roasted malt, 

TypeError: 'NoneType' object is not iterable

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/pymongo/mongo_client.py", line 1333, in _process_periodic_tasks
    cursor_ids, address, topology, session=None)
  File "/opt/conda/lib/python3.6/site-packages/pymongo/mongo_client.py", line 1263, in _kill_cursors
    server = topology.select_server_by_address(tuple(address))
  File "/opt/conda/lib/python3.6/site-packages/pymongo/topology.py", line 247, in select_server_by_address
    address)
  File "/opt/conda/lib/python3.6/site-packages/pymongo/topology.py", line 224, in select_server
    address))
  File "/opt/conda/lib/python3.6/site-packages/pymongo/topology.py", line 183, in select_servers
    selector, server_timeout, address)
  File "/opt/conda/lib/python3.6/site-packages/pymongo/topology.py", line 199, in _select_servers_loop
    self._error_message(selector))
pymongo.errors.ServerSelectionTimeoutError: 54.149.252.88:27016: timed out


## Converting PyMongo info to a dataframe.  

In [11]:
from pymongo import MongoClient
client = MongoClient('250.250.250.250',20000)

# All the keys in the json shit
Beers['Beer_final'].find_one({}).keys()

dict_keys(['_id', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'Beer_Name', 'Brewery_Name', 'ABV', 'Type', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'])

In [33]:
# creating a dataframe to test
df = pd.DataFrame(data = None, columns  = Beers['Beer_final'].find_one({}).keys())
df = df.append(Beers['Beer_final'].find_one({}), ignore_index =True)

In [34]:
df.shape

(1, 32)

In [17]:
# verifying with the one tests that things went where it needed to go.
df['Beer_Name']

0    Indian Brown Ale
Name: Beer_Name, dtype: object

In [22]:
# lets try adding another
df = df.append(Beers['Beer_final'].find({})[1], ignore_index=True)

In [31]:
df['32']

0                                                  NaN
1    A: Very, very dark with a medium brown colored...
Name: 32, dtype: object

In [36]:

beer_df = pd.DataFrame(data = None, columns  = Beers['Beer_final'].find_one({}).keys())

for beer in Beers['Beer_final'].find({}):
    beer_df = beer_df.append(beer, ignore_index=True)

In [37]:
# check shape for removal of duplicates
beer_df.shape

(10384, 53)

In [38]:
# save DB with removed entries
beer_df.to_csv('Beers.csv', index = False, encoding = 'utf-8')